In [0]:
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


Instructions for updating:
non-resource variables are not supported in the long term


**In this file i have shown the comparison between Diuble DDQN and DQN. **

In [0]:
env_name = "CartPole-v0"
env = gym.make(env_name)

In [0]:
#main aim for ddqn is avoid selecting the maximum value too early 
# ---> ddqn helps us to select which actions and state to take which are present in local and arget neural network
class QNetwork():
    def __init__(self, state_dim, action_size, tau=0.01):
        tf.compat.v1.reset_default_graph()
        self.state_in = tf.placeholder(tf.float32, shape=[None, *state_dim])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.q_target_in = tf.placeholder(tf.float32, shape=[None])
        action_one_hot = tf.one_hot(self.action_in, depth=action_size)
        
        self.q_state_local = self.build_model(action_size, "local") #calling function for local q-state
        self.q_state_target = self.build_model(action_size, "target") #calling function for target q-state
        
        self.q_state_action = tf.reduce_sum(tf.multiply(self.q_state_local, action_one_hot), axis=1)
        self.loss = tf.reduce_mean(tf.square(self.q_state_action - self.q_target_in))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss) #optimizer is adam optimizer; iteratively updating network weights, in training data
        #accesing the network through tensorflow function
        self.local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="local")
        self.target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="target") #specifying the keys for target network
        self.updater = tf.group([tf.assign(t, t + tau*(l-t)) for t,l in zip(self.target_vars, self.local_vars)]) #update operation with group function
           
    #defining the function if the target value is true(boolean), else we can use local network.        
    def get_q_state(self, session, state, use_target=False):
        q_state_op = self.q_state_target if use_target else self.q_state_local
        q_state = session.run(q_state_op, feed_dict={self.state_in: state})
        return q_state       

     #dense layer, of target network, the layers are same as neural network, but params are different
    def build_model(self, action_size, scope):
        with tf.variable_scope(scope): #seprating the scope
            hidden1 = tf.layers.dense(self.state_in, 100, activation=tf.nn.relu)
            q_state = tf.layers.dense(hidden1, action_size, activation=None)
            return q_state
               
    def update_model(self, session, state, action, q_target):
        feed = {self.state_in: state, self.action_in: action, self.q_target_in: q_target}
        session.run([self.optimizer, self.updater], feed_dict=feed)
     

In [0]:
class Buffering():
    def add(self, experience):
        self.buffer.append(experience)   
               
    def __init__(self, maxlen):
        self.buffer = deque(maxlen=maxlen)
             
    def sample(self, batch_size):
        sample_size = min(len(self.buffer), batch_size)
        samples = random.choices(self.buffer, k=sample_size)
        return map(list, zip(*samples))

 

In [0]:

class DoubleDQNAgent():
    def __init__(self, env):
        self.state_dim = env.observation_space.shape
        self.action_size = env.action_space.n
        self.q_network = QNetwork(self.state_dim, self.action_size)
        self.replay_buffer = Buffering(maxlen=10000)
        self.gamma = 0.97
        self.eps = 1.0
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def train(self, state, action, next_state, reward, done, use_DDQN=True): #we have to define our q-targets and specify whether we are using ddqn or not
        self.replay_buffer.add((state, action, next_state, reward, done))
        states, actions, next_states, rewards, dones = self.replay_buffer.sample(50)  
        next_actions = np.argmax(self.q_network.get_q_state(self.sess, next_states, use_target=False), axis=1) #we get the arguments of the target nets from the local nets that we are using
        q_next_states = self.q_network.get_q_state(self.sess, next_states, use_target=use_DDQN) #over here you can turn on and off, whether you want to use ddqn or not
        q_next_states[dones] = np.zeros([self.action_size])
        q_next_states_next_actions = q_next_states[np.arange(next_actions.shape[0]), next_actions] #next action and next state of the target net
        q_targets = rewards + self.gamma * q_next_states_next_actions
        self.q_network.update_model(self.sess, states, actions, q_targets)
        
        if done: self.eps = max(0.1, 0.99*self.eps)

    def get_action(self, state):
        q_state = self.q_network.get_q_state(self.sess, [state])
        action_greedy = np.argmax(q_state)
        action_random = np.random.randint(self.action_size)
        action = action_random if random.random() < self.eps else action_greedy
        return action
    
    def __del__(self):
        self.sess.close()

In [0]:
num_runs = 50
run_rewards = []
for n in range(num_runs):
    print("Run {}".format(n))
    ep_rewards = []
    agent = None #since each agent creates a tensorflow session, we need define it none, it would delete all the agents
    agent = DoubleDQNAgent(env)
    num_episodes = 500

    for ep in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False
        while not done:
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            agent.train(state, action, next_state, reward, done, use_DDQN=(n%2==0)) #over here we are running all the ddqn on every even run, to show the difference between dqn and ddqn
            #env.render()
            total_reward += reward
            state = next_state
        ep_rewards.append(total_reward) #saving all the rewards to a list          
    run_rewards.append(ep_rewards)

Run 0
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Run 10
Run 11
Run 12
Run 13
Run 14
Run 15
Run 16
Run 17
Run 18
Run 19
Run 20
Run 21
Run 22
Run 23
Run 24
Run 25
Run 26
Run 27
Run 28
Run 29
Run 30
Run 31
Run 32
Run 33
Run 34
Run 35
Run 36
Run 37
Run 38
Run 39
Run 40
Run 41
Run 42
Run 43
Run 44
Run 45
Run 46
Run 47
Run 48
Run 49


In [1]:
#visualiing the difference ddqn vs dqn in the performance
import matplotlib.pyplot as plt
%matplotlib inline

for n, ep_rewards in enumerate(run_rewards):
    x = range(len(ep_rewards))
    cumsum = np.cumsum(ep_rewards) #average rewards for every episodes
    avgs = [cumsum[ep]/(ep+1) if ep<100 else (cumsum[ep]-cumsum[ep-200])/200 for ep in x]
    col = "r" if (n%2==0) else "b"
    plt.plot(x, avgs, color=col)
    
plt.title("DQN vs DDQN performance")
plt.xlabel("Episodes")
plt.ylabel("Last 200 episode average rewards")
plt.legend()
#down in the graph we can see the differnce in the performance in ddqn and dqn, where ddqn are giving faster cumuliative rewards than dqn in fewer episodes

NameError: ignored

In [0]:
env.close()